# ASR Assignment 2019-20

This notebook has been provided as a template to get you started on the assignment.  Feel free to use it for your development, or do your development directly in Python.

You can find a full description of the assignment [here](http://www.inf.ed.ac.uk/teaching/courses/asr/2019-20/coursework.pdf).

You are provided with two Python modules `observation_model.py` and `wer.py`.  The first was described in [Lab 3](https://github.com/Ore-an/asr_lab3/blob/master/asr_lab3.ipynb).  The second can be used to compute the number of substitution, deletion and insertion errors between ASR output and a reference text.

It can be used as follows:

```python
import wer

my_refence = 'A B C'
my_output = 'A C C D'

wer.compute_alignment_errors(my_reference, my_output)
```

This produces a tuple $(s,d,i)$ giving counts of substitution,
deletion and insertion errors respectively - in this example (1, 0, 1).  The function accepts either two strings, as in the example above, or two lists.  Matching is case sensitive.

## Template code

Assuming that you have already made a function to generate an WFST, `create_wfst()` and a decoder class, `MyViterbiDecoder`, you can perform recognition on all the audio files as follows:


# Help function to parse the lexicon and generate symbol tables

In [11]:
import numpy as np
import dill

In [12]:
dill.load_session('notebook_env.db')
import openfst_python as fst


ModuleNotFoundError: No module named 'openfst_python'

In [ ]:

def parse_lexicon(lex_file):
    """
    Parse the lexicon file and return it in dictionary form.
    
    Args:
        lex_file (str): filename of lexicon file with structure '<word> <phone1> <phone2>...'
                        eg. peppers p eh p er z

    Returns:
        lex (dict): dictionary mapping words to list of phones
    """
    
    lex = {}  # create a dictionary for the lexicon entries (this could be a problem with larger lexica)
    with open(lex_file, 'r') as f:
        for line in f:
            line = line.split()  # split at each space
            lex[line[0]] = line[1:]  # first field the word, the rest is the phones
    return lex

def generate_symbol_tables(lexicon, n=3):
    '''
    Return word, phone and state symbol tables based on the supplied lexicon
        
    Args:
        lexicon (dict): lexicon to use, created from the parse_lexicon() function
        n (int): number of states for each phone HMM
        
    Returns:
        word_table (fst.SymbolTable): table of words
        phone_table (fst.SymbolTable): table of phones
        state_table (fst.SymbolTable): table of HMM phone-state IDs
    '''
    
    state_table = fst.SymbolTable()
    phone_table = fst.SymbolTable()
    word_table = fst.SymbolTable()
    
    # add empty <eps> symbol to all tables
    state_table.add_symbol('<eps>')
    phone_table.add_symbol('<eps>')
    word_table.add_symbol('<eps>')
    
    for word, phones  in lexicon.items():
        
        word_table.add_symbol(word)
        
        for p in phones: # for each phone
            
            phone_table.add_symbol(p)
            for i in range(1,n+1): # for each state 1 to n
                state_table.add_symbol('{}_{}'.format(p, i))
            
    return word_table, phone_table, state_table


# call these two functions
lex = parse_lexicon('lexicon.txt')
word_table, phone_table, state_table = generate_symbol_tables(lex)



# Generate phone, word wfst for any sequence of phones and words

In [ ]:
def generate_phone_wfst(f, start_state, phone, n):
    """
    Generate a WFST representating an n-state left-to-right phone HMM
    
    Args:
        f (fst.Fst()): an FST object, assumed to exist already
        start_state (int): the index of the first state, assmed to exist already
        phone (str): the phone label 
        n (int): number of states for each phone HMM
        
    Returns:
        the final state of the FST
    """
    
    current_state = start_state
    
    for i in range(1, n+1):
        
        in_label = state_table.find('{}_{}'.format(phone, i))
        
        # self-loop back to current state
        f.add_arc(current_state, fst.Arc(in_label, 0, None, current_state))
        
        # transition to next state
        
        # we want to output the phone label on the final state
        # note: if outputting words instead this code should be modified
        if i == n:
            out_label = phone_table.find(phone)
        else:
            out_label = 0   # output empty <eps> label
            
        next_state = f.add_state()
        f.add_arc(current_state, fst.Arc(in_label, out_label, None, next_state))    
       
        current_state = next_state
        
    return current_state

def generate_word_wfst(word):
    """ Generate a WFST for any word in the lexicon, composed of 3-state phone WFSTs.
        This will currently output word labels.  
        Exercise: could you modify this function and the one above to output a single phone label instead?
    
    Args:
        word (str): the word to generate
        
    Returns:
        the constructed WFST
    
    """
    f = fst.Fst()
    
    # create the start state
    start_state = f.add_state()
    f.set_start(start_state)
    
    current_state = start_state
    
    # iterate over all the phones in the word
    for phone in lex[word]:   # will raise an exception if word is not in the lexicon
        
        current_state = generate_phone_wfst(f, current_state, phone, 3)
    
        # note: new current_state is now set to the final state of the previous phone WFST
        
    f.set_final(current_state)
    
    return f
  
def generate_phone_recognition_wfst():
    """ generate a HMM to recognise any sequence phone in the lexicon (assumes 3-state HMMs per phone)
    
    Returns:
        the constructed WFST
    
    """
       
    # first, get a list of all the phones in the lexicon
    # there are lots of way to do this.  Here, we use the set() object

    # will contain all unique phones in the lexicon
    phone_set = set()
    
    for pronunciation in lex.values():
        phone_set = phone_set.union(pronunciation)

    f = fst.Fst()
    
    # create a single start state
    start_state = f.add_state()
    f.set_start(start_state)
    
    for phone in phone_set:
        
        # we need to add an empty arc from the start state to where the actual phone HMM
        # will begin.  If you can't see why this is needed, try without it!
        current_state = f.add_state()
        f.add_arc(start_state, fst.Arc(0, 0, None, current_state))
    
        end_state = generate_phone_wfst(f, current_state, phone, 3)
        
        f.set_final(end_state)
        f.add_arc(end_state, fst.Arc(0,0,None,start_state))
    return f

def generate_word_recognition_wfst():
    """ Generate a WFST for any word in the lexicon, composed of 3-state phone WFSTs.
        This will currently output word labels.  
        Exercise: could you modify this function and the one above to output a single phone label instead?
    
    Args:
        word (str): the word to generate
        
    Returns:
        the constructed WFST
    
    """
    f = fst.Fst()
    
    # create the start state
    start_state = f.add_state()
    f.set_start(start_state)
    
    current_state = start_state
    
    lex_set = []
    for word in lex.keys():
        lex_set.append(word)
    for word in lex_set: 
        # iterate over all the phones in the word
        current_state = f.add_state()
        f.add_arc(start_state, fst.Arc(0, 0, None, current_state))    
        for phone in lex[word]:   # will raise an exception if word is not in the lexicon
        
            current_state = generate_phone_wfst(f, current_state, phone, 3)

            # note: new current_state is now set to the final state of the previous phone WFST
        f.set_final(current_state)
        f.add_arc(current_state, fst.Arc(0,0,None,start_state))
        current_state=start_state
    f.set_input_symbols(state_table)
    f.set_output_symbols(phone_table)
    return f


In [ ]:

lex = parse_lexicon('lexicon.txt')
word_table, phone_table, state_table = generate_symbol_tables(lex)  # we won't use state_table in this lab

def generate_L_wfst(lex):
    """ express the lexicon in WFST form
    
    Args:
        lexicon (dict): lexicon to use, created from the parse_lexicon() function
    
    Returns:
        the constructed lexicon WFST
    
    """
    L = fst.Fst()
    
    # create a single start state
    start_state = L.add_state()
    L.set_start(start_state)
    
    for (word, pron) in lex.items():
        current_state = start_state
        for (i,phone) in enumerate(pron):
            next_state = L.add_state()
            if i == len(pron)-1:
                # add word output symbol on the final arc
                L.add_arc(current_state, fst.Arc(phone_table.find(phone), \
                                                 word_table.find(word), None, next_state))
            else:
                L.add_arc(current_state, fst.Arc(phone_table.find(phone),0, None, next_state))
            
            current_state = next_state
                          
        L.set_final(current_state)
        
    L.set_input_symbols(phone_table)
    L.set_output_symbols(word_table)                      
    
    return L

L = generate_L_wfst(lex)
L.arcsort()



In [ ]:
def generate_L_wfst_modified(lex):
    """ express the lexicon in WFST form
    
    Args:
        lexicon (dict): lexicon to use, created from the parse_lexicon() function
    
    Returns:
        the constructed lexicon WFST
    
    """
    L = fst.Fst()
    
    # create a single start state
    start_state = L.add_state()
    L.set_start(start_state)
    
    for (word, pron) in lex.items():
        current_state = start_state
        for (i,phone) in enumerate(pron):
            next_state = L.add_state()
            if i == len(pron)-1:
                # add word output symbol on the final arc
                L.add_arc(current_state, fst.Arc(phone_table.find(phone), \
                                                 word_table.find(word), None, next_state))
            else:
                L.add_arc(current_state, fst.Arc(phone_table.find(phone),0, None, next_state))
            
            current_state = next_state
                          
        L.set_final(current_state)
        L.add_arc(current_state,fst.Arc(0,0,None,start_state))
        
    L.set_input_symbols(phone_table)
    L.set_output_symbols(word_table)                      
    
    return L

L = generate_L_wfst_modified(lex)
L.arcsort()



In [ ]:
def generate_linear_phone_wfst(phone_list):
    
    P = fst.Fst()
    
    current_state = P.add_state()
    P.set_start(current_state)
    
    for p in phone_list:
        next_state = P.add_state()
        P.add_arc(current_state, fst.Arc(phone_table.find(p), phone_table.find(p), None, next_state))
        current_state = next_state
        
    P.set_final(current_state)
    P.set_input_symbols(phone_table)
    P.set_output_symbols(phone_table)
    return P

In [ ]:
def generate_word_from_phone(phone_sequence,f,current_state):
    
    if (len(phone_sequence) == 0):
        return None
    
    if (len(phone_sequence) == 1):
        for arc in f.arcs(current_state):
            if (phone_sequence[0] == f.input_symbols().find(arc.ilabel)):
                if (arc.olabel != 0):
                    return f.output_symbols().find(arc.olabel)
                for arc2 in f.arcs(arc.nextstate):
                    if (arc2.ilabel == 0):
                        return f.output_symbols().find(arc2.olabel)
        return None
    
    for arc in f.arcs(current_state):
        if (phone_sequence[0] == f.input_symbols().find(arc.ilabel)):
            if (generate_word_from_phone_ND(phone_sequence[1:],f,arc.nextstate) != None):
                return generate_word_from_phone_ND(phone_sequence[1:],f,arc.nextstate)

In [ ]:
import observation_model
import math

class MyViterbiDecoder:
    
    NLL_ZERO = 1e10  # define a constant representing -log(0).  This is really infinite, but approximate
                     # it here with a very large number
    
    def __init__(self, f, audio_file_name):
        """Set up the decoder class with an audio file and WFST f
        """
        self.om = observation_model.ObservationModel()
        self.f = f
        
        if audio_file_name:
            self.om.load_audio(audio_file_name)
        else:
            self.om.load_dummy_audio()
        
        self.initialise_decoding()

        
    def initialise_decoding(self):
        """set up the values for V_j(0) (as negative log-likelihoods)
        
        """
        
        self.V = []   # stores likelihood along best path reaching state j
        self.B = []   # stores identity of best previous state reaching state j
        self.W = []   # stores output labels sequence along arc reaching j - this removes need for 
                      # extra code to read the output sequence along the best path
        
        for t in range(self.om.observation_length()+1):
            self.V.append([self.NLL_ZERO]*self.f.num_states())
            self.B.append([-1]*self.f.num_states())
            self.W.append([[] for i in range(self.f.num_states())])  #  multiplying the empty list doesn't make multiple
        
        # The above code means that self.V[t][j] for t = 0, ... T gives the Viterbi cost
        # of state j, time t (in negative log-likelihood form)
        # Initialising the costs to NLL_ZERO effectively means zero probability    
        
        # give the WFST start state a probability of 1.0   (NLL = 0.0)
        self.V[0][self.f.start()] = 0.0
        
        # some WFSTs might have arcs with epsilon on the input (you might have already created 
        # examples of these in earlier labs) these correspond to non-emitting states, 
        # which means that we need to process them without stepping forward in time.  
        # Don't worry too much about this!  
        self.traverse_epsilon_arcs(0)        
        
    def traverse_epsilon_arcs(self, t):
        """Traverse arcs with <eps> on the input at time t
        
        These correspond to transitions that don't emit an observation
        
        We've implemented this function for you as it's slightly trickier than
        the normal case.  You might like to look at it to see what's going on, but
        don't worry if you can't fully follow it.
        
        """
        n = 0
        states_to_traverse = list(self.f.states()) # traverse all states
        while states_to_traverse:
            
            # Set i to the ID of the current state, the first 
            # item in the list (and remove it from the list)
            i = states_to_traverse.pop(0)   
        
            # don't bother traversing states which have zero probability
            if self.V[t][i] == self.NLL_ZERO:
                    continue
        
            for arc in f.arcs(i):
                
                if arc.ilabel == 0:     # if <eps> transition
                  
                    j = arc.nextstate   # ID of next state  
                
                    if self.V[t][j] > self.V[t][i] + float(arc.weight):
                        
                        # this means we've found a lower-cost path to
                        # state j at time t.  We might need to add it
                        # back to the processing queue.
                        self.V[t][j] = self.V[t][i] + float(arc.weight)
                        n=n+1
                        # save backtrace information.  In the case of an epsilon transition, 
                        # we save the identity of the best state at t-1.  This means we may not
                        # be able to fully recover the best path, but to do otherwise would
                        # require a more complicated way of storing backtrace information
                        self.B[t][j] = self.B[t][i] 
                        
                        # and save the output labels encountered - this is a list, because
                        # there could be multiple output labels (in the case of <eps> arcs)
                        if arc.olabel != 0:
                            self.W[t][j] = self.W[t][i] + [arc.olabel]
                        else:
                            self.W[t][j] = self.W[t][i]
                        
                        if j not in states_to_traverse:
                            states_to_traverse.append(j)

        return n
    
    def forward_step(self, t):
        
        n = 0
        
        for i in self.f.states():
            
            if not self.V[t-1][i] == self.NLL_ZERO:   # no point in propagating states with zero probability
                
                for arc in self.f.arcs(i):
                    
                    if arc.ilabel != 0: # <eps> transitions don't emit an observation
                        j = arc.nextstate
                        tp = float(arc.weight)  # transition prob
                        ep = -self.om.log_observation_probability(self.f.input_symbols().find(arc.ilabel), t)  # emission negative log prob
                        prob = tp + ep + self.V[t-1][i] # they're logs
                        n = n+1
                        if prob < self.V[t][j]:
                            self.V[t][j] = prob
                            self.B[t][j] = i
                            
                            # store the output labels encountered too
                            if arc.olabel !=0:
                                self.W[t][j] = [arc.olabel]
                            else:
                                self.W[t][j] = []
        return n           
    
    def finalise_decoding(self):
        """ this incorporates the probability of terminating at each state
        """
        n = 0
        for state in self.f.states():
            final_weight = float(self.f.final(state))
            if self.V[-1][state] != self.NLL_ZERO:
                if final_weight == math.inf:
                    self.V[-1][state] = self.NLL_ZERO  # effectively says that we can't end in this state
                else:
                    self.V[-1][state] += final_weight
                    n=n+1
        # get a list of all states where there was a path ending with non-zero probability
        finished = [x for x in self.V[-1] if x < self.NLL_ZERO]
        if not finished:  # if empty
            print("No path got to the end of the observations.")
        return n
        
    def decode(self):
        n = 0
        self.initialise_decoding()
        t = 1
        while t <= self.om.observation_length():
            n+=self.forward_step(t)
            n+=self.traverse_epsilon_arcs(t)
            t += 1
        n+=self.finalise_decoding()
        return n
    
    def backtrace(self):
        
        best_final_state = self.V[-1].index(min(self.V[-1])) # argmin
        best_state_sequence = [best_final_state]
        best_out_sequence = []
        
        t = self.om.observation_length()   # ie T
        j = best_final_state
        
        while t >= 0:
            i = self.B[t][j]
            best_state_sequence.append(i)
            best_out_sequence = self.W[t][j] + best_out_sequence  # computer scientists might like
                                                                                # to make this more efficient!

            # continue the backtrace at state i, time t-1
            j = i  
            t-=1
        
        best_state_sequence.reverse()
        # convert the best output sequence from FST integer labels into strings
        best_out_sequence = ' '.join([ self.f.output_symbols().find(label) for label in best_out_sequence])
        best_out_sequence = best_out_sequence.split()
        return (best_state_sequence, output_words_sequence(best_out_sequence))

In [13]:
f = generate_word_recognition_wfst()
L = fst.determinize(generate_L_wfst_modified(lex))


NameError: name 'fst' is not defined

In [ ]:
def num_arcs_and_states(f):
    return sum(1 + f.num_arcs(s) for s in f.states())
def num_arcs(f):
    return sum(f.num_arcs(s) for s in f.states())

In [ ]:
print ('number of states and args word_recognition_wfst are {},{},and in sum is {}'.format(f.num_states(),num_arcs(f),num_arcs_and_states(f)))
print ('number of states and args lexicon wfst are {},{},and in sum is {}'.format(L.num_states(),num_arcs(L),num_arcs_and_states(L)))


In [ ]:
def output_words_sequence(phone_sequence):
    P = fst.compose(generate_linear_phone_wfst(phone_sequence),L).project(project_output=True).rmepsilon()
    current_state = P.start()
    words = []
    for n in range(P.num_states()-1):
        for arc in P.arcs(current_state):
            words.append(P.input_symbols().find(arc.ilabel))
            current_state = arc.nextstate
    return words

# Task 1 :baseline

In [ ]:
import timeit
import glob
import os
import wer
import observation_model
import openfst_python as fst
# ... (add your code to create WFSTs and Viterbi Decoder)

def read_transcription(wav_file):
    """
    Get the transcription corresponding to wav_file.
    """
    
    transcription_file = os.path.splitext(wav_file)[0] + '.txt'
    
    with open(transcription_file, 'r') as f:
        transcription = f.readline().strip()
    
    return transcription


#f = create_wfst()



#Your statements here



start = timeit.default_timer()
errors = []
words = []
times_decode = []
times_backtrace = []
fc_num = []
for wav_file in glob.glob('/group/teaching/asr/labs/recordings/*.wav'):
    
    decoder = MyViterbiDecoder(f, wav_file)
    start_decode = timeit.default_timer()
    fc_num.append(decoder.decode())
    stop_decode = timeit.default_timer()
    times_decode.append(stop_decode - start_decode)
    print('Time: ', stop_decode - start_decode)  
    start_backtrace = timeit.default_timer()    
    (state_path, words) = decoder.backtrace()  # you'll need to modify the backtrace() from Lab 4
                                               # to return the words along the best path
    stop_backtrace = timeit.default_timer()
    times_backtrace.append(stop_backtrace - start_backtrace)
    print('Time: ', stop_backtrace - start_backtrace)  
    transcription = read_transcription(wav_file)
    error_counts = wer.compute_alignment_errors(transcription, words)
    word_count = len(transcription.split())
    errors.append(error_counts)
    words.append(word_count)
    print(error_counts, word_count)     # you'll need to accumulate these to produce an overall Word Error Rate



In [ ]:
print (fc_num)

In [ ]:
np.sum(fc_num)

In [ ]:
np.mean(fc_num)

In [ ]:
np.std(fc_num)

# Experiment

System Turning

In [ ]:
dill.dump_session('notebook_env.db')